In [1]:
#selected Final Working Code
# Step 1: Install required packages in Google Colab
!pip install streamlit pandas openai xlsxwriter pillow
!npm install -g localtunnel


# Step 2: Write the Streamlit app code
ai_examiner_code = """
import os
import streamlit as st
import pandas as pd
import xlsxwriter
import google.generativeai as genai

# Securely configure the Gemini API with your API key
genai.configure(api_key='YOUR_API_KEY')

# Admin Authentication
def admin_login():
    st.title("Admin Login")
    username = st.text_input("Username")
    password = st.text_input("Password", type='password')

    if st.button("Login"):
        if username == 'admin' and password == 'admin':
            st.success("Login Successful")
            return True
        else:
            st.error("Incorrect Username/Password")
            return False
    return False

# Admin Console
def admin_console():
    st.title("Admin Console")
    st.write("Enter the number of questions you want to create:")
    num_questions = st.number_input('Number of Questions', min_value=1, step=1)

    if 'temp_questions' not in st.session_state:
        st.session_state.temp_questions = [''] * num_questions
        st.session_state.temp_answers = [''] * num_questions

    if st.button("Generate Q&A Form"):
        st.session_state.temp_questions = [''] * num_questions
        st.session_state.temp_answers = [''] * num_questions

    if len(st.session_state.temp_questions) == num_questions:
        for i in range(int(num_questions)):
            st.session_state.temp_questions[i] = st.text_input(f'Question {i+1}', key=f'q_{i}')
            st.session_state.temp_answers[i] = st.text_input(f'Answer {i+1}', key=f'a_{i}')

    if st.button("Save Questions"):
        if '' not in st.session_state.temp_questions and '' not in st.session_state.temp_answers:
            st.session_state['questions'] = st.session_state.temp_questions
            st.session_state['answers'] = st.session_state.temp_answers
            st.success("Questions and Answers have been saved successfully!")
        else:
            st.error("Please complete all questions and answers before saving.")

# Student Exam Portal
def student_portal():
    st.title("Student Exam Portal")
    st.write("Please fill in your details below:")

    name = st.text_input("Name")
    email = st.text_input("Email")
    reg_no = st.text_input("Registration Number")

    if name and email and reg_no:
        st.write("Answer the following questions:")

        if 'questions' not in st.session_state or 'answers' not in st.session_state:
            st.error("No questions have been set by the admin yet.")
            return

        if 'student_answers' not in st.session_state:
            st.session_state.student_answers = [''] * len(st.session_state['questions'])

        for i, question in enumerate(st.session_state['questions']):
            st.write(f"Q{i+1}: {question}")
            st.session_state.student_answers[i] = st.text_input(f'Your Answer for Question {i+1}', key=f'student_a_{i}')

        if st.button("Submit"):
            if '' not in st.session_state.student_answers:
                relevance_scores, explanations, cumulative_score = evaluate_answers(st.session_state.student_answers)
                student_data = {
                    'Name': name,
                    'Email': email,
                    'Registration Number': reg_no,
                }
                save_to_excel(student_data, relevance_scores, explanations, cumulative_score)

                # Display AI-evaluated scores and explanations
                st.write("### AI-Evaluated Scores and Explanations")
                for i, (score, explanation) in enumerate(zip(relevance_scores, explanations)):
                    st.write(f"**Explanation for Question {i+1}:**")
                    st.write(f"Relevance Score: {score}/100")
                    st.write(f"{explanation}")
                    st.write("---")

                # Display the total percentage obtained
                st.write(f"**Total Percentage Obtained: {cumulative_score:.2f}%**")
            else:
                st.error("Please answer all questions before submitting.")

# Evaluate Answers Using Gemini API
def evaluate_answers(student_answers):
    relevance_scores = []
    explanations = []
    total_score = 0
    model = genai.GenerativeModel("gemini-1.5-flash")

    for i, answer in enumerate(student_answers):
        prompt = (
            f"Evaluate the relevance of this student's answer: '{answer}' to the correct answer: '{st.session_state['answers'][i]}'. "
            f"Provide a relevance score out of 100 and a brief explanation."
        )
        response = model.generate_content(prompt)
        response_text = response.text.strip()

        try:
            # Parse the relevance score (assumes the score is near the end)
            score_part = response_text.split("Relevance Score:")[-1]
            score_value = int(''.join(filter(str.isdigit, score_part.split('/')[0])))

            # Parse the explanation (assumes explanation follows "Explanation:")
            explanation = response_text.split("Explanation:")[-1].strip()

            relevance_scores.append(score_value)
            explanations.append(explanation)
            total_score += score_value
        except (IndexError, ValueError) as e:
            relevance_scores.append(0)
            explanations.append(f"Failed to parse the response correctly: {e}")  # Detailed error for debugging

    cumulative_score = total_score / len(student_answers) if student_answers else 0
    return relevance_scores, explanations, cumulative_score

# Save Data to Excel
def save_to_excel(student_data, relevance_scores, explanations, cumulative_score):
    # Append relevance scores, explanations, and cumulative score to student data
    for i, (score, explanation) in enumerate(zip(relevance_scores, explanations)):
        student_data[f'Q{i+1} Score (%)'] = score
        student_data[f'Q{i+1} Explanation'] = explanation
    student_data['Total Score (%)'] = cumulative_score

    df = pd.DataFrame([student_data])
    file_path = '/content/students_data.xlsx'

    if not os.path.exists(file_path):
        with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
            df.to_excel(writer, sheet_name='Exam Results', index=False)
    else:
        existing_df = pd.read_excel(file_path, sheet_name='Exam Results')
        df = pd.concat([existing_df, df], ignore_index=True)
        with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
            df.to_excel(writer, sheet_name='Exam Results', index=False)

    st.success("Student data has been saved to Excel!")

# Main Application Logic
def main():
    st.set_page_config(page_title="AI Examiner", layout="centered")

    if 'authenticated' not in st.session_state:
        st.session_state['authenticated'] = False

    if st.session_state['authenticated'] or admin_login():
        st.session_state['authenticated'] = True
        menu = ["Admin Console", "Student Portal"]
        choice = st.sidebar.selectbox("Menu", menu)

        if choice == "Admin Console":
            admin_console()
        elif choice == "Student Portal":
            student_portal()

if __name__ == "__main__":
    main()


"""

# Step 3: Write the Streamlit app code to a Python script file
with open("ai_examiner_app.py", "w") as text_file:
    text_file.write(ai_examiner_code)

# Step 4: Run the Streamlit app with localtunnel
!streamlit run ai_examiner_app.py & npx localtunnel --port 8501


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully 